<a href="https://colab.research.google.com/github/AZoyaV/Python/blob/main/A_B_testing_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. ПІДГОТОВКА СЕРЕДОВИЩА ТА ДАНИХ

In [ ]:
from google.colab import drive
import pandas as pd
drive.mount("/content/drive")
%cd /content/drive/MyDrive/bq-results-abtest

# Завантажуємо дані
data = pd.read_csv("bq-results-abtest.csv")
display(data.head(10))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/bq-results-abtest


,date,country,device,continent,channel,test,test_group,event_name,value
0,2020-11-01,Lithuania,mobile,Europe,Organic Search,2,2,new account,1
1,2020-11-01,Lithuania,desktop,Europe,Paid Search,1,2,new account,1
2,2020-11-02,North Macedonia,desktop,Europe,Direct,1,1,new account,1
3,2020-11-02,Georgia,desktop,Asia,Organic Search,2,2,new account,1
4,2020-11-02,Kenya,desktop,Africa,Paid Search,2,2,new account,1
5,2020-11-03,Cambodia,desktop,Asia,Organic Search,1,2,new account,1
6,2020-11-03,Ecuador,mobile,Americas,Direct,1,2,new account,1
7,2020-11-03,Qatar,mobile,Asia,Social Search,2,2,new account,1
8,2020-11-03,Estonia,mobile,Europe,Direct,1,1,new account,1
9,2020-11-03,Jordan,desktop,Asia,Paid Search,1,1,new account,1


In [ ]:
# Перевіряємо назви
data["event_name"].unique()

array(['new account', 'session', 'session_with_orders', 'scroll',
       'page_view', 'view_promotion', 'view_item', 'user_engagement',
       'session_start', 'add_shipping_info', 'first_visit',
       'begin_checkout', 'add_to_cart', 'view_search_results',
       'add_payment_info', 'select_item', 'select_promotion', 'click',
       'view_item_list'], dtype=object)

In [ ]:
# Визначаємо метрики, які ми будемо аналізувати
metrics = ["add_payment_info", "add_shipping_info", "begin_checkout", "new account"]

# Визначаємо знаменник для розрахунку конверсії
event_denominator = "session"

# Фільтруємо дані, залишаючи тільки потрібні метрики та знаменник
data_metrics = data[data["event_name"].isin(metrics + ["session"])].copy()

# Перевіримо результат
data_metrics[["country", "test", "test_group", "event_name", "value"]].head(10)

,country,test,test_group,event_name,value
0,Lithuania,2,2,new account,1
1,Lithuania,1,2,new account,1
2,North Macedonia,1,1,new account,1
3,Georgia,2,2,new account,1
4,Kenya,2,2,new account,1
5,Cambodia,1,2,new account,1
6,Ecuador,1,2,new account,1
7,Qatar,2,2,new account,1
8,Estonia,1,1,new account,1
9,Jordan,1,1,new account,1


## 2. ФУНКЦІЯ ДЛЯ A/B-ТЕСТУВАННЯ З СЕГМЕНТАЦІЄЮ ТА Z-КРИТЕРІЄМ

In [ ]:
import statsmodels.api as sm

def ab_test_two_groups(data_metrics):
    result = []

    # Отримуємо всі унікальні ідентифікатори тестів із колонки "test"
    test_list = data_metrics["test"].unique()

    # Зовнішній цикл: Ітерація по кожному A/B-тесту
    for test_number in test_list:
        df_test = data_metrics[data_metrics["test"] == test_number]

        # Беремо всі країни в межах цього тесту
        countries = df_test[df_test["country"] != "(not set)"]["country"].unique()

        #Середній цикл: Ітерація по кожній країні (сегментація)
        for country in countries:
            df_country = df_test[df_test["country"] == country]

            # Отримуємо всі унікальні метрики, окрім denominator
            metrics_list = [m for m in df_country["event_name"].unique() if m != event_denominator]

            # Внутрішній цикл: Ітерація по кожній метриці конверсії
            for metric in metrics_list:
                # Контрольна група (group 1)
                db_control = df_country[df_country["test_group"] == 1]
                control_event = db_control[db_control["event_name"] == metric]["value"].sum()
                control_denominator = db_control[db_control["event_name"] == event_denominator]["value"].sum()
                conv_control = (control_event / control_denominator * 100) if control_denominator > 0 else 0

                # Тестова група (group 2)
                db_test_group = df_country[df_country["test_group"] == 2]
                test_event = db_test_group[db_test_group["event_name"] == metric]["value"].sum()
                test_denominator = db_test_group[db_test_group["event_name"] == event_denominator]["value"].sum()
                conv_test = (test_event / test_denominator * 100) if test_denominator > 0 else 0

                # Зміна метрики
                metric_change = (conv_test - conv_control) / max(conv_control, 1e-6) * 100

                # Z-test, що порівнює дві пропорції (коефіцієнти конверсії)
                z_stat, p_value = sm.stats.proportions_ztest(
                    [control_event, test_event],
                    [control_denominator, test_denominator]
                )

                # Визначення значущості
                # Якщо p-value < 0.05, ми відхиляємо Нульову гіпотезу і різниця є статистично значущою.
                significant_status = p_value < 0.05

                # Робимо список
                result.append({
                    "test_number": test_number,
                    "country": country,
                    "metric": metric,
                    "numerator": metric,
                    "denominator": event_denominator,
                    "numerator_control": control_event,
                    "denominator_control": control_denominator,
                    "conversion_rate_control": round(conv_control, 3),
                    "numerator_test": test_event,
                    "denominator_test": test_denominator,
                    "conversion_rate_test": round(conv_test, 3),
                    "metric_change_%": round(metric_change, 3),
                    "z_stat": z_stat,
                    "p_value": p_value,
                    "significant": significant_status
                })

    return pd.DataFrame(result)


# Виклик функції
ab_country_df_two_groups = ab_test_two_groups(data_metrics)
display(ab_country_df_two_groups.head(10))

/usr/local/lib/python3.12/dist-packages/statsmodels/stats/proportion.py:1024: RuntimeWarning: invalid value encountered in sqrt
  std_diff = np.sqrt(var_)


,test_number,country,metric,numerator,denominator,numerator_control,denominator_control,conversion_rate_control,numerator_test,denominator_test,conversion_rate_test,metric_change_%,z_stat,p_value,significant
0,2,Lithuania,new account,new account,session,7,69,10.145,9,62,14.516,4.308800e+01,-0.762865,0.445544,False
1,2,Lithuania,add_shipping_info,add_shipping_info,session,1,69,1.449,0,62,0.000,-1.000000e+02,0.951558,0.341321,False
2,2,Lithuania,begin_checkout,begin_checkout,session,1,69,1.449,0,62,0.000,-1.000000e+02,0.951558,0.341321,False
3,2,Georgia,new account,new account,session,0,25,0.000,2,27,7.407,7.407407e+08,-1.387777,0.165205,False
4,2,Georgia,begin_checkout,begin_checkout,session,4,25,16.000,0,27,0.000,-1.000000e+02,2.163331,0.030516,True
5,2,Georgia,add_payment_info,add_payment_info,session,3,25,12.000,0,27,0.000,-1.000000e+02,1.854284,0.063699,False
6,2,Georgia,add_shipping_info,add_shipping_info,session,4,25,16.000,0,27,0.000,-1.000000e+02,2.163331,0.030516,True
7,2,Kenya,new account,new account,session,1,34,2.941,2,27,7.407,1.518520e+02,-0.801222,0.423003,False
8,2,Kenya,add_shipping_info,add_shipping_info,session,7,34,20.588,1,27,3.704,-8.201100e+01,1.940405,0.052331,False
9,2,Kenya,begin_checkout,begin_checkout,session,10,34,29.412,1,27,3.704,-8.740700e+01,2.594025,0.009486,True


## Наданий код виконує сегментований A/B-аналіз для двох груп (test_group: 1 та 2), використовуючи коефіцієнт конверсії як основну метрику, та визначає статистичну значущість за допомогою Z-test.

## 1. Підготовка даних та Ітерація (Цикли)
Код використовує три рівні циклів для перевірки кожного сегмента:

- for test_number in test_list: аналіз розпочинається з ітерації по кожному унікальному A/B-тесту (test).
- for country in countries: далі результати сегментуються за країною.
- for metric in metrics_list: внутрішній цикл перебирає кожну цільову подію (add_payment_info, begin_checkout тощо).

## 2. Розрахунок Конверсії (CR)
Для кожної комбінації (Тест-Країна-Метрика) виконуються паралельні розрахунки для обох груп:
- Чисельник (_event): Сума значень події-метрики для даної групи в даній країні.
- Знаменник (_denominator): Сума значень події-знаменника (session) для даної групи в даній країні.
- Коефіцієнт конверсії (conv_)

Далі обчислюється зміна метрики у відсотках (metric_change_%), яка показує, наскільки конверсія Тестової групи краща або гірша за Контрольну групу.

## 3. Логіка Статистичної Значущості (Z-критерій)
Логіка визначення значущості базується на p-value
- Z-test перевіряє Нульову гіпотезу (H_0): що не існує реальної різниці між конверсією Контрольної та Тестової груп, і будь-яка спостережувана різниця є випадковою.
- Код використовує порогове значення alpha = 0.05$ (5%).
- Якщо p-value < 0.05, то ймовірність випадкової різниці дуже низька. Отже, ми відхиляємо Нульову гіпотезу. Різниця вважається статистично значущою (significant - True), і ми можемо довіряти результату тесту.
- Якщо p-value > 0.05, то ймовірність випадкової різниці висока. Ми не відхиляємо Нульову гіпотезу. Результат вважається статистично не значущим (significant - False), і ми не можемо стверджувати, що зміна дала ефект.

In [ ]:
ab_country_df_two_groups.to_csv("ab_test_two_groups.csv", index=False)

from google.colab import files
files.download("ab_test_two_groups.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Файл csv для обчислень: https://drive.google.com/file/d/1tpNVU-ngfZTpK4d6gMrySSMqmH4dmjzJ/view?usp=sharing

Файл з фінальним результатом: https://drive.google.com/file/d/1x1_MaTN_SrLtjZPwUNaGOE3Z6lhC5qtF/view?usp=sharing

Tableau: https://public.tableau.com/app/profile/zoya.ashmarina/viz/ABTest_17602133014230/ABtest?publish=yes